In [1]:
import cv2
import numpy as np
import win32gui, win32ui, win32con, win32api
import time
import ctypes
import copy as cp

In [2]:
#Function for getting a screenshot
def grab_screen(region=None):

    hwin = win32gui.GetDesktopWindow()

    if region:
            left,top,x2,y2 = region
            width = x2 - left + 1
            height = y2 - top + 1
    else:
        width = win32api.GetSystemMetrics(win32con.SM_CXVIRTUALSCREEN)
        height = win32api.GetSystemMetrics(win32con.SM_CYVIRTUALSCREEN)
        left = win32api.GetSystemMetrics(win32con.SM_XVIRTUALSCREEN)
        top = win32api.GetSystemMetrics(win32con.SM_YVIRTUALSCREEN)

    hwindc = win32gui.GetWindowDC(hwin)
    srcdc = win32ui.CreateDCFromHandle(hwindc)
    memdc = srcdc.CreateCompatibleDC()
    bmp = win32ui.CreateBitmap()
    bmp.CreateCompatibleBitmap(srcdc, width, height)
    memdc.SelectObject(bmp)
    memdc.BitBlt((0, 0), (width, height), srcdc, (left, top), win32con.SRCCOPY)
    
    signedIntsArray = bmp.GetBitmapBits(True)
    img = np.fromstring(signedIntsArray, dtype='uint8')
    img.shape = (height,width,4)

    srcdc.DeleteDC()
    memdc.DeleteDC()
    win32gui.ReleaseDC(hwin, hwindc)
    win32gui.DeleteObject(bmp.GetHandle())
    return cv2.cvtColor(img, cv2.COLOR_BGRA2RGB)

In [3]:
#Functions for controlling the game
SendInput = ctypes.windll.user32.SendInput

PUL = ctypes.POINTER(ctypes.c_ulong)
class KeyBdInput(ctypes.Structure):
    _fields_ = [("wVk", ctypes.c_ushort),
                ("wScan", ctypes.c_ushort),
                ("dwFlags", ctypes.c_ulong),
                ("time", ctypes.c_ulong),
                ("dwExtraInfo", PUL)]

class HardwareInput(ctypes.Structure):
    _fields_ = [("uMsg", ctypes.c_ulong),
                ("wParamL", ctypes.c_short),
                ("wParamH", ctypes.c_ushort)]

class MouseInput(ctypes.Structure):
    _fields_ = [("dx", ctypes.c_long),
                ("dy", ctypes.c_long),
                ("mouseData", ctypes.c_ulong),
                ("dwFlags", ctypes.c_ulong),
                ("time",ctypes.c_ulong),
                ("dwExtraInfo", PUL)]

class Input_I(ctypes.Union):
    _fields_ = [("ki", KeyBdInput),
                 ("mi", MouseInput),
                 ("hi", HardwareInput)]

class Input(ctypes.Structure):
    _fields_ = [("type", ctypes.c_ulong),
                ("ii", Input_I)]
    
    
def KeyEvent(flags):
    extra = ctypes.c_ulong(0)
    ii_ = Input_I()
    ii_.ki = KeyBdInput( 0, hexKeyCode, flags, 0, ctypes.pointer(extra) )
    x = Input( ctypes.c_ulong(1), ii_ )
    ctypes.windll.user32.SendInput(1, ctypes.pointer(x), ctypes.sizeof(x))
    
def PressKey(hexKeyCode):
    KeyEvent(0x0008)
    
def ReleaseKey(hexKeyCode):
    KeyEvent(0x0008 | 0x0002)
    
def MouseEvent(flags,moveX=0,moveY=0):
    extra = ctypes.c_ulong(0)
    ii_ = Input_I()
    ii_.mi = MouseInput( moveX,moveY, 0, flags , 0, ctypes.pointer(extra) )
    x = Input( ctypes.c_ulong(0), ii_ )
    ctypes.windll.user32.SendInput(1, ctypes.pointer(x), ctypes.sizeof(x))
    
def MoveMouse(moveX,moveY):
    MouseEvent(0x001,moveX,moveY)

def MouseClick():
    MouseEvent(0x002)
    
def MouseRelease():
    MouseEvent(0x004)

In [4]:
#Functions for getting input from the game
keyList = ["\b"]
for char in "ABCDEFGHIJKLMNOPQRSTUVWXYZ 123456789,.'APS$/\\":
    keyList.append(char)

def key_check():
    keys = []
    for key in keyList:
        if win32api.GetAsyncKeyState(ord(key)):
            keys.append(key)
    return keys

def isclicked():
    return win32api.GetKeyState(0x01) < 0

In [9]:
def try_grab_screen(seconds=10): #for testing that everything's working
    t0=time.time()
    while True:
            screen = grab_screen()
            screen = cv2.resize(screen,(0,0),fx=0.25,fy=0.25)
            cv2.imshow('window',screen)

            if (time.time()-t0 > seconds) or (cv2.waitKey(25) & 0xFF == ord('q')):
                cv2.destroyAllWindows()
                break
    print('grab_screen successful')

In [11]:
try_grab_screen()

grab_screen successful


In [12]:
def nothing(x):
    pass

cv2.namedWindow('window')

# create trackbars for color change
cv2.createTrackbar('H','window',0,360,nothing)
cv2.createTrackbar('S','window',0,255,nothing)
cv2.createTrackbar('V','window',0,255,nothing)
cv2.createTrackbar('H2','window',0,360,nothing)
cv2.createTrackbar('S2','window',0,255,nothing)
cv2.createTrackbar('V2','window',0,255,nothing)

last_time = time.time()
counter=0
is_active=False
while True:
        screen = grab_screen(region=(0,30,1000,600))
        time_ammount=time.time()-last_time
        print('Frame took {} seconds'.format(time_ammount))
        last_time = time.time()      
        screen = cv2.resize(screen,(0,0),fx=0.33,fy=0.33)
        
        H=cv2.getTrackbarPos('H','window')
        S=cv2.getTrackbarPos('S','window')
        V=cv2.getTrackbarPos('V','window')
        H2=cv2.getTrackbarPos('H2','window')
        S2=cv2.getTrackbarPos('S2','window')
        V2=cv2.getTrackbarPos('V2','window')
        
        #lower_color=np.array([H,S,V])   For tunning values in execution time
        #upper_color=np.array([H2,S2,V2]) For tunning values in execution time
        lower_color=np.array([115,80,180]) #Found values
        upper_color=np.array([125,255,255]) ##Found values
        screen = cv2.cvtColor(screen, cv2.COLOR_BGR2RGB)
        
        screen = cv2.cvtColor(screen, cv2.COLOR_RGB2HSV)
        
        screen = cv2.inRange(screen, lower_color, upper_color)
        screen = cv2.GaussianBlur(screen, (3,3), 0 )
        screen = cv2.dilate(screen,np.ones((5,5),np.uint8),iterations = 1)
        
        aux = cp.copy(screen)
        _, contours ,_ = cv2.findContours(aux,cv2.RETR_TREE,cv2.CHAIN_APPROX_SIMPLE)
        C=[]
        for cnt in contours:
            contarea = cv2.contourArea(cnt)
            if contarea > 15:
                x,y,w,h=cv2.boundingRect(cnt)
                if w < 1.5*h and h < 1.5*w:
                    dx,dy=int(3*(x+w/2)-500),int(3*(y+h/2)-300)
                    if dx < 300 and dy<100:
                        d=(dx**2)+(dy**2)
                        C.append((cnt,dx,dy,d,contarea,(x,y,w,h)))
           
            
        if len(C)>1:        
            C= sorted(C,key=lambda tup: tup[3])[:2]
            C= sorted(C,key=lambda tup: tup[4], reverse = True)[:2]
            cnt,dx,dy,_,contarea,rect=C[0]
            x,y,w,h=rect
            screen=cv2.rectangle(screen,(x,y),(x+w,y+h),255)
            if isclicked() and is_active:
                MoveMouse(dx,dy)
            if 'T' in key_check() and counter<=0:
                is_active=not is_active
                counter=0.5
                
            elif counter>0:
                counter-=time_ammount
        
        cv2.imshow('window',screen)
        

        if cv2.waitKey(25) & 0xFF == ord('q'):
            cv2.destroyAllWindows()
            break
        

Frame took 0.015633821487426758 seconds
Frame took 0.8162777423858643 seconds
Frame took 0.1000206470489502 seconds
Frame took 0.06851005554199219 seconds
Frame took 0.07846593856811523 seconds
Frame took 0.10014200210571289 seconds
Frame took 0.06853842735290527 seconds
Frame took 0.10076284408569336 seconds
Frame took 0.05288267135620117 seconds
Frame took 0.0625157356262207 seconds
Frame took 0.06950712203979492 seconds
Frame took 0.06653738021850586 seconds
Frame took 0.09576749801635742 seconds
Frame took 0.0672304630279541 seconds
Frame took 0.10374069213867188 seconds
Frame took 0.06452751159667969 seconds
Frame took 0.06853318214416504 seconds
Frame took 0.07845830917358398 seconds
Frame took 0.06893444061279297 seconds
Frame took 0.08379244804382324 seconds
Frame took 0.06846952438354492 seconds
Frame took 0.07854938507080078 seconds
Frame took 0.06902861595153809 seconds
Frame took 0.11590909957885742 seconds
Frame took 0.06886053085327148 seconds
Frame took 0.100300550460815